In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow  as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Conv1D, Flatten, MaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
%matplotlib inline

Using TensorFlow backend.


In [2]:
## Creating five_fold_validation test
def five_fold_split(X,Y):
    step = (len(X)//5)
    E, L, ET, LT = [],[],[],[]
    indexes = set(list(range(len(X))))
    for i in range(5):
        if i < 4:
            LT.append(Y[i*step:(i+1)*step])
            ET.append(X[i*step:(i+1)*step])
            E.append(X[list(indexes.difference(list(range(i*step,(i+1)*step))))])
            L.append(Y[list(indexes.difference(list(range(i*step,(i+1)*step))))])
        else:
            LT.append(Y[i*step:])
            ET.append(X[i*step:])
            E.append(X[:i*step])
            L.append(Y[:i*step])

    return E,L, ET, LT

In [3]:
#Acquiring the dataset from CSV file
df = pd.read_csv('../spam.csv',delimiter=',',encoding='latin-1')
df.head()

#Dropping last three columns, (since the dataset has the last three columns filled with nan values)
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

labels = df.v1.to_numpy()
ham = sum(labels=="ham")
print("Ham: " +  str(ham)+ " Spam: " + str(5572-ham))

X = df.v2
Y = df.v1
#Encoding labels
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)
#Splitting the dataset into train and test set (85%-15%)
E, L, ET, LT = five_fold_split(X,Y)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
Ham: 4825 Spam: 747


REVERSE

In [11]:
Erev, ETrev = [],[]
for i in range(5):
    Erev.append(E[i].map(lambda x : ' '.join(x.split()[::-1])))
    ETrev.append(ET[i].map(lambda x : ' '.join(x.split()[::-1])))

In [12]:
max_words = 1000
max_len = 150

In [13]:
Etokseqrev, ETtokseqrev = [],[]
for fold in range(5):
    tok = Tokenizer(num_words=max_words)
    seq = tok.fit_on_texts(Erev[fold])
    sequencesTrain = tok.texts_to_sequences(Erev[fold])
    sequencesTest = tok.texts_to_sequences(ETrev[fold])
    Etokseqrev.append(sequence.pad_sequences(sequencesTrain,maxlen=max_len))
    ETtokseqrev.append(sequence.pad_sequences(sequencesTest,maxlen=max_len))

In [ ]:
#Tokenizing and padding each fold in train and test set
Etokseq, ETtokseq = [],[]
for fold in range(5):
    tok = Tokenizer(num_words=max_words)
    seq = tok.fit_on_texts(E[fold])
    sequencesTrain = tok.texts_to_sequences(E[fold])
    sequencesTest = tok.texts_to_sequences(ET[fold])
    Etokseq.append(sequence.pad_sequences(sequencesTrain,maxlen=max_len))
    ETtokseq.append(sequence.pad_sequences(sequencesTest,maxlen=max_len))

In [5]:
#defining lists of hyperparameters
optimizers = [tf.keras.optimizers.RMSprop, tf.keras.optimizers.Adam]
losses = [tf.keras.losses.MSE, tf.keras.losses.binary_crossentropy]
batch_sizes = [16,32,64,128]
learning_rates = [0.005, 0.001, 0.0005]

In [14]:
#Recurrent Neural Network 1
def RNN_std():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [7]:
#Recurrent Neural Network 2
def RNN2():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,25,input_length=max_len)(inputs)
    layer = LSTM(32)(layer)
    layer = Dense(64,name='FC1')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(128,activation='relu',name='FC2')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(32,activation='relu',name='FC3')(layer)
    layer = Dropout(0.1)(layer)
    layer = Dense(1,activation='relu',name='FC4')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model


In [ ]:
#Recurrent Neural Network 3
def RNN3():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,100,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dropout(0.1)(layer)
    layer = Dense(1,activation='relu',name='FC2')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [7]:
#Neural Network + Embedding - LSTM
def NN1():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [11]:
#Neural Network - Embedding - LSTM
def NN2():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Dense(256,name='FC1')(inputs)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model


In [6]:
results = []

In [13]:
#For each optimizer, loss, batch size and learning rate listed above, this code performs training
#and evaluation of each fold in order to obtain the 5 fold cross validation results
import numpy as np
model = None
with open('resultsNN2', 'w') as f:
    for optimizer in optimizers:
        for loss in losses:
            for bs in batch_sizes:
                for lr in learning_rates:
                    train_accuracies = np.zeros(5)
                    test_accuracies = np.zeros(5)
                    for fold in range(5):
                        model = NN2()
                        model.compile(loss=loss,optimizer=optimizer(learning_rate=lr),metrics=['accuracy'])
                        history = model.fit(Etokseq[fold],L[fold],batch_size=bs,epochs=10,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
                        train_accuracies[fold] = history.history['accuracy'][-1]
                        test_accuracies[fold] = model.evaluate(ETtokseq[fold],LT[fold])[1]
                        del model
                    mean_train = np.mean(train_accuracies)
                    mean_test = np.mean(test_accuracies)
                    string = "BatchSize: " + str(bs) + " Loss:" + str(loss) + " Optimizer:" + str(optimizer)+ " LearningRate: " + str(lr) + " TrainAcc: " + str(mean_train) + " TestAcc: " + str(mean_test) +"\n"
                    print(string)
                    results.append(string)
                    f.write(string)

Epoch 1/10
223/223 [==============================] - 0s 2ms/step - loss: 0.1336 - accuracy: 0.8665 - val_loss: 0.1291 - val_accuracy: 0.8711
Epoch 2/10
223/223 [==============================] - 0s 1ms/step - loss: 0.1301 - accuracy: 0.8699 - val_loss: 0.1289 - val_accuracy: 0.8711

In [12]:
del model

NameError: name 'model' is not defined

Naive Bayes training and 5-fold cross validation

In [ ]:
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

train_accuracies = np.zeros(5)
test_accuracies = np.zeros(5)
for fold in range(5):
    Naive = naive_bayes.MultinomialNB()
    Naive.fit(Etokseq[fold],L[fold])
    # predict the labels on validation dataset
    predictions_Train_NB = Naive.predict(Etokseq[fold])
    predictions_Test_NB = Naive.predict(ETtokseq[fold])
    # Use accuracy_score function to get the accuracy
    train_accuracies[fold] = accuracy_score(predictions_Train_NB ,L[fold])*100
    test_accuracies[fold] = accuracy_score(predictions_Test_NB ,LT[fold])*100
    print("Naive Bayes Accuracy Train Score -> ",train_accuracies[fold])
    print("Naive Bayes Accuracy Test Score -> ",test_accuracies[fold])
    del Naive
mean_train = np.mean(train_accuracies)
mean_test = np.mean(test_accuracies)


print("Mean Train Accuracy: " + str(mean_train))
print("Mean Test Accuracy: " + str(mean_test))

In [8]:
del Naive

Support Vector Machine training and 5-fold cross validation

In [13]:
from sklearn import  svm
train_accuracies = np.zeros(5)
test_accuracies = np.zeros(5)
for fold in range(5):
    SVM = svm.SVC(C=1000, gamma=0.1)
    SVM.fit(Etokseq[fold],L[fold])
    # predict the labels on validation dataset
    predictions_Train_NB = SVM.predict(Etokseq[fold])
    predictions_Test_NB = SVM.predict(ETtokseq[fold])
    # Use accuracy_score function to get the accuracy
    train_accuracies[fold] = accuracy_score(predictions_Train_NB ,L[fold])*100
    test_accuracies[fold] = accuracy_score(predictions_Test_NB ,LT[fold])*100
    print("SVM Accuracy Train Score -> ",train_accuracies[fold])
    print("SVM Accuracy Test Score -> ",test_accuracies[fold])
    del SVM
mean_train = np.mean(train_accuracies)
mean_test = np.mean(test_accuracies)


print("Mean Train Accuracy: " + str(mean_train))
print("Mean Test Accuracy: " + str(mean_test))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/loc

SVM Accuracy Train Score ->  99.9775684163302
SVM Bayes Accuracy Test Score ->  90.12567324955117
SVM Accuracy Train Score ->  99.9775684163302
SVM Bayes Accuracy Test Score ->  91.92100538599641
SVM Accuracy Train Score ->  99.9775684163302
SVM Bayes Accuracy Test Score ->  92.01077199281868
SVM Accuracy Train Score ->  100.0
SVM Bayes Accuracy Test Score ->  90.66427289048474
SVM Accuracy Train Score ->  99.97755834829444
SVM Bayes Accuracy Test Score ->  91.66666666666666
Mean Train Accuracy: 99.98205271945702
Mean Test Accuracy: 91.27767803710353


K-Nearest Neighbors training and 5-fold cross validation

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

train_accuracies = np.zeros(5)
test_accuracies = np.zeros(5)
for fold in range(5):
    KNN = KNeighborsClassifier(n_neighbors=5)
    KNN.fit(Etokseq[fold],L[fold])
    # predict the labels on validation dataset
    predictions_Train_NB = KNN.predict(Etokseq[fold])
    predictions_Test_NB = KNN.predict(ETtokseq[fold])
    # Use accuracy_score function to get the accuracy
    train_accuracies[fold] = accuracy_score(predictions_Train_NB ,L[fold])*100
    test_accuracies[fold] = accuracy_score(predictions_Test_NB ,LT[fold])*100
    print("KNN Accuracy Train Score -> ",train_accuracies[fold])
    print("KNN Accuracy Test Score -> ",test_accuracies[fold])
    del KNN
mean_train = np.mean(train_accuracies)
mean_test = np.mean(test_accuracies)


print("Mean Train Accuracy: " + str(mean_train))
print("Mean Test Accuracy: " + str(mean_test))

/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8:

Naive Bayes Accuracy Train Score ->  91.99192462987888
Naive Bayes Accuracy Test Score ->  87.61220825852783
Naive Bayes Accuracy Train Score ->  91.13952445042621
Naive Bayes Accuracy Test Score ->  88.24057450628366
Naive Bayes Accuracy Train Score ->  91.74517720951098
Naive Bayes Accuracy Test Score ->  90.84380610412927
Naive Bayes Accuracy Train Score ->  92.21624046657693
Naive Bayes Accuracy Test Score ->  87.52244165170556
Naive Bayes Accuracy Train Score ->  91.65170556552962
Naive Bayes Accuracy Test Score ->  89.69534050179212
Mean Train Accuracy: 91.74891446438453
Mean Test Accuracy: 88.78287420448768


Decision Tree Classifier training and 5-fold cross validation

In [10]:
from sklearn.tree import DecisionTreeClassifier
train_accuracies = np.zeros(5)
test_accuracies = np.zeros(5)
for fold in range(5):
    DTC = DecisionTreeClassifier()
    DTC.fit(Etokseq[fold],L[fold])
    # predict the labels on validation dataset
    predictions_Train_NB = DTC.predict(Etokseq[fold])
    predictions_Test_NB = DTC.predict(ETtokseq[fold])
    # Use accuracy_score function to get the accuracy
    train_accuracies[fold] = accuracy_score(predictions_Train_NB ,L[fold])*100
    test_accuracies[fold] = accuracy_score(predictions_Test_NB ,LT[fold])*100
    print("DTC Accuracy Train Score -> ",train_accuracies[fold])
    print("DTC Accuracy Test Score -> ",test_accuracies[fold])
    del DTC
mean_train = np.mean(train_accuracies)
mean_test = np.mean(test_accuracies)


print("Mean Train Accuracy: " + str(mean_train))
print("Mean Test Accuracy: " + str(mean_test))

DTC Accuracy Train Score ->  99.9775684163302
DTC Accuracy Test Score ->  88.50987432675045
DTC Accuracy Train Score ->  99.9775684163302
DTC Accuracy Test Score ->  89.22800718132855
DTC Accuracy Train Score ->  99.9775684163302
DTC Accuracy Test Score ->  91.11310592459606
DTC Accuracy Train Score ->  100.0
DTC Accuracy Test Score ->  87.97127468581688
DTC Accuracy Train Score ->  99.97755834829444
DTC Accuracy Test Score ->  88.88888888888889
Mean Train Accuracy: 99.98205271945702
Mean Test Accuracy: 89.14223020147617


Random Forest Classifier training and 5-fold cross validation

In [11]:
from sklearn.ensemble import RandomForestClassifier
train_accuracies = np.zeros(5)
test_accuracies = np.zeros(5)
for fold in range(5):
    RFC = RandomForestClassifier()
    RFC.fit(Etokseq[fold],L[fold])
    # predict the labels on validation dataset
    predictions_Train_NB = RFC.predict(Etokseq[fold])
    predictions_Test_NB = RFC.predict(ETtokseq[fold])
    # Use accuracy_score function to get the accuracy
    train_accuracies[fold] = accuracy_score(predictions_Train_NB ,L[fold])*100
    test_accuracies[fold] = accuracy_score(predictions_Test_NB ,LT[fold])*100
    print("RFC Accuracy Train Score -> ",train_accuracies[fold])
    print("RFC Accuracy Test Score -> ",test_accuracies[fold])
    del RFC
mean_train = np.mean(train_accuracies)
mean_test = np.mean(test_accuracies)


print("Mean Train Accuracy: " + str(mean_train))
print("Mean Test Accuracy: " + str(mean_test))

/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: Dat

RFC Accuracy Train Score ->  99.9775684163302
RFC Accuracy Test Score ->  92.63913824057451
RFC Accuracy Train Score ->  99.9775684163302
RFC Accuracy Test Score ->  93.35727109515261
RFC Accuracy Train Score ->  99.9775684163302
RFC Accuracy Test Score ->  94.3447037701975
RFC Accuracy Train Score ->  100.0
RFC Accuracy Test Score ->  92.36983842010773
RFC Accuracy Train Score ->  99.95511669658886
RFC Accuracy Test Score ->  93.81720430107528
Mean Train Accuracy: 99.97756438911588
Mean Test Accuracy: 93.30563116542152


Word2Vec experiments

In [24]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
text_corpus = list()
for sent in E[0].values:
    text_corpus.append(sent.split())
#text word2vec
model = Word2Vec(text_corpus, min_count=5, size=max_len, workers=4)
filename = 'embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)

In [7]:
def load_embedding(filename):
	# load embedding into memory, skip first line
	file = open(filename,'r')
	lines = file.readlines()[1:]
	file.close()
	# create a map of words to vectors
	embedding = dict()
	for line in lines:
		parts = line.split()
		# key is string word, value is numpy array for vector
		embedding[parts[0]] = np.asarray(parts[1:], dtype='float32')
	return embedding

In [8]:
# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
	# total vocabulary size plus 0 for unknown words
	vocab_size = len(vocab) + 1
	# define weight matrix dimensions with all 0
	weight_matrix = np.zeros((vocab_size, 100))
	# step vocab, store vectors using the Tokenizer's integer mapping
	for word, i in vocab.items():
		weight_matrix[i] = embedding.get(word)
	return weight_matrix

In [9]:

embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [10]:
def get_emb_layer(fold):
    tokenizer = Tokenizer(nb_words=max_words)
    tokenizer.fit_on_texts(fold)
    sequences = tokenizer.texts_to_sequences(fold)
    word_index = tokenizer.word_index
    embedding_matrix = np.zeros((len(word_index) + 1, 100))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    embedding_layer = Embedding(len(word_index) + 1,
                                100,
                                weights=[embedding_matrix],
                                input_length=max_len,
                                trainable=False)
    return embedding_layer

In [11]:
def RNN_W2V(emb_layer):
    # define model
    inputs = Input(name='inputs',shape=[max_len])
    layer = emb_layer(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    modelW2Vec = Model(inputs=inputs,outputs=layer)
    return modelW2Vec

In [12]:
model = None
with open('resultsRNNW2V', 'w') as f:
    for optimizer in optimizers:
        for loss in losses:
            for bs in batch_sizes:
                for lr in learning_rates:
                    train_accuracies = np.zeros(5)
                    test_accuracies = np.zeros(5)
                    for fold in range(5):
                        model = RNN_W2V(get_emb_layer(E[fold]))
                        model.compile(loss=loss,optimizer=optimizer(learning_rate=lr),metrics=['accuracy'])
                        history = model.fit(Etokseq[fold],L[fold],batch_size=bs,epochs=10,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
                        train_accuracies[fold] = history.history['accuracy'][-1]
                        test_accuracies[fold] = model.evaluate(ETtokseq[fold],LT[fold])[1]
                        del model
                    mean_train = np.mean(train_accuracies)
                    mean_test = np.mean(test_accuracies)
                    string = "BatchSize: " + str(bs) + " Loss:" + str(loss) + " Optimizer:" + str(optimizer)+ " LearningRate: " + str(lr) + " TrainAcc: " + str(mean_train) + " TestAcc: " + str(mean_test) +"\n"
                    print(string)
                    results.append(string)
                    f.write(string)

/home/andrea/.local/lib/python3.6/site-packages/keras_preprocessing/text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Epoch 1/10
223/223 [==============================] - 13s 57ms/step - loss: 0.0432 - accuracy: 0.9459 - val_loss: 0.0184 - val_accuracy: 0.9731
Epoch 2/10
223/223 [==============================] - 14s 63ms/step - loss: 0.0206 - accuracy: 0.9739 - val_loss: 0.0163 - val_accuracy: 0.9832
Epoch 3/10
35/35 [==============================] - 1s 19ms/step - loss: 0.0091 - accuracy: 0.9901
Epoch 1/10
223/223 [==============================] - 13s 58ms/step - loss: 0.0495 - accuracy: 0.9391 - val_loss: 0.0276 - val_accuracy: 0.9664
Epoch 2/10
223/223 [==============================] - 13s 58ms/step - loss: 0.0198 - accuracy: 0.9770 - val_loss: 0.0219 - val_accuracy: 0.9753
Epoch 3/10
223/223 [==============================] - 12s 55ms/step - loss: 0.0147 - accuracy: 0.9832 - val_loss: 0.0171 - val_accuracy: 0.9787
Epoch 4/10
223/223 [==============================] - 12s 54ms/step - loss: 0.0116 - accuracy: 0.9854 - val_loss: 0.0119 - val_accuracy: 0.9843
Epoch 5/10
223/223 [=================

35/35 [==============================] - 1s 16ms/step - loss: 0.0310 - accuracy: 0.9597
BatchSize: 16 Loss:<function mean_squared_error at 0x7feba67cf1e0> Optimizer:<class 'tensorflow.python.keras.optimizer_v2.rmsprop.RMSprop'> LearningRate: 0.0005 TrainAcc: 0.9782364249229432 TestAcc: 0.9687757015228271

Epoch 1/10
112/112 [==============================] - 7s 65ms/step - loss: 0.0520 - accuracy: 0.9316 - val_loss: 0.1003 - val_accuracy: 0.8520
Epoch 2/10
112/112 [==============================] - 7s 61ms/step - loss: 0.0264 - accuracy: 0.9678 - val_loss: 0.0172 - val_accuracy: 0.9776
Epoch 3/10
112/112 [==============================] - 7s 61ms/step - loss: 0.0186 - accuracy: 0.9776 - val_loss: 0.0160 - val_accuracy: 0.9809
Epoch 4/10
112/112 [==============================] - 7s 61ms/step - loss: 0.0143 - accuracy: 0.9826 - val_loss: 0.0153 - val_accuracy: 0.9798
Epoch 5/10
112/112 [==============================] - 7s 61ms/step - loss: 0.0098 - accuracy: 0.9893 - val_loss: 0.0090 -

35/35 [==============================] - 1s 16ms/step - loss: 0.0712 - accuracy: 0.9066
Epoch 1/10
112/112 [==============================] - 7s 64ms/step - loss: 0.0592 - accuracy: 0.9291 - val_loss: 0.0222 - val_accuracy: 0.9720
Epoch 2/10
112/112 [==============================] - 7s 61ms/step - loss: 0.0247 - accuracy: 0.9708 - val_loss: 0.0172 - val_accuracy: 0.9776
Epoch 3/10
112/112 [==============================] - 7s 61ms/step - loss: 0.0193 - accuracy: 0.9773 - val_loss: 0.0151 - val_accuracy: 0.9832
Epoch 4/10
112/112 [==============================] - 7s 60ms/step - loss: 0.0167 - accuracy: 0.9809 - val_loss: 0.0148 - val_accuracy: 0.9843
Epoch 5/10
35/35 [==============================] - 1s 16ms/step - loss: 0.0185 - accuracy: 0.9794
Epoch 1/10
112/112 [==============================] - 7s 63ms/step - loss: 0.0586 - accuracy: 0.9288 - val_loss: 0.0257 - val_accuracy: 0.9697
Epoch 2/10
35/35 [==============================] - 1s 16ms/step - loss: 0.0463 - accuracy: 0.9425

35/35 [==============================] - 1s 16ms/step - loss: 0.0249 - accuracy: 0.9668
Epoch 1/10
56/56 [==============================] - 5s 85ms/step - loss: 0.0550 - accuracy: 0.9318 - val_loss: 0.0561 - val_accuracy: 0.9238
Epoch 2/10
56/56 [==============================] - 5s 81ms/step - loss: 0.0268 - accuracy: 0.9680 - val_loss: 0.0245 - val_accuracy: 0.9709
Epoch 3/10
56/56 [==============================] - 5s 81ms/step - loss: 0.0196 - accuracy: 0.9776 - val_loss: 0.0183 - val_accuracy: 0.9765
Epoch 4/10
35/35 [==============================] - 1s 16ms/step - loss: 0.0238 - accuracy: 0.9668
BatchSize: 64 Loss:<function mean_squared_error at 0x7feba67cf1e0> Optimizer:<class 'tensorflow.python.keras.optimizer_v2.rmsprop.RMSprop'> LearningRate: 0.001 TrainAcc: 0.9773955345153809 TestAcc: 0.97038893699646

Epoch 1/10
56/56 [==============================] - 5s 85ms/step - loss: 0.0856 - accuracy: 0.8954 - val_loss: 0.0286 - val_accuracy: 0.9697
Epoch 2/10
35/35 [===============

...

223/223 [==============================] - 11s 51ms/step - loss: 0.0627 - accuracy: 0.9820 - val_loss: 0.0720 - val_accuracy: 0.9798
Epoch 4/10
223/223 [==============================] - 11s 51ms/step - loss: 0.0578 - accuracy: 0.9834 - val_loss: 0.0672 - val_accuracy: 0.9798
Epoch 5/10
223/223 [==============================] - 11s 51ms/step - loss: 0.0488 - accuracy: 0.9832 - val_loss: 0.0606 - val_accuracy: 0.9821
Epoch 6/10
35/35 [==============================] - 1s 16ms/step - loss: 0.0853 - accuracy: 0.9758
BatchSize: 16 Loss:<function binary_crossentropy at 0x7feba67cfe18> Optimizer:<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'> LearningRate: 0.0005 TrainAcc: 0.9837341666221618 TestAcc: 0.9752331018447876

Epoch 1/10
112/112 [==============================] - 7s 64ms/step - loss: 0.1562 - accuracy: 0.9456 - val_loss: 0.0587 - val_accuracy: 0.9832
Epoch 2/10
112/112 [==============================] - 7s 60ms/step - loss: 0.0727 - accuracy: 0.9784 - val_loss: 0.0466 - v

35/35 [==============================] - 1s 17ms/step - loss: 0.0939 - accuracy: 0.9695
BatchSize: 32 Loss:<function binary_crossentropy at 0x7feba67cfe18> Optimizer:<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'> LearningRate: 0.001 TrainAcc: 0.9841816663742066 TestAcc: 0.9748762845993042

Epoch 1/10
112/112 [==============================] - 7s 63ms/step - loss: 0.2727 - accuracy: 0.9021 - val_loss: 0.0896 - val_accuracy: 0.9776
Epoch 2/10
112/112 [==============================] - 7s 60ms/step - loss: 0.1043 - accuracy: 0.9692 - val_loss: 0.0845 - val_accuracy: 0.9697
Epoch 3/10
112/112 [==============================] - 7s 60ms/step - loss: 0.0859 - accuracy: 0.9731 - val_loss: 0.0703 - val_accuracy: 0.9809
Epoch 4/10
112/112 [==============================] - 7s 60ms/step - loss: 0.0678 - accuracy: 0.9809 - val_loss: 0.0614 - val_accuracy: 0.9787
Epoch 5/10
112/112 [==============================] - 7s 60ms/step - loss: 0.0669 - accuracy: 0.9795 - val_loss: 0.0606 - val_a

56/56 [==============================] - 4s 78ms/step - loss: 0.0704 - accuracy: 0.9787 - val_loss: 0.0623 - val_accuracy: 0.9776
Epoch 3/10
35/35 [==============================] - 1s 16ms/step - loss: 0.0627 - accuracy: 0.9803
BatchSize: 64 Loss:<function binary_crossentropy at 0x7feba67cfe18> Optimizer:<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'> LearningRate: 0.005 TrainAcc: 0.9914173245429992 TestAcc: 0.9816946864128113

Epoch 1/10
56/56 [==============================] - 5s 83ms/step - loss: 0.3085 - accuracy: 0.8836 - val_loss: 0.1408 - val_accuracy: 0.9339
Epoch 2/10
56/56 [==============================] - 4s 78ms/step - loss: 0.1156 - accuracy: 0.9619 - val_loss: 0.1044 - val_accuracy: 0.9608
Epoch 3/10
56/56 [==============================] - 4s 79ms/step - loss: 0.0867 - accuracy: 0.9717 - val_loss: 0.0656 - val_accuracy: 0.9821
Epoch 4/10
56/56 [==============================] - 4s 78ms/step - loss: 0.0674 - accuracy: 0.9784 - val_loss: 0.0639 - val_accuracy: 0

Epoch 3/10
28/28 [==============================] - 3s 95ms/step - loss: 0.0657 - accuracy: 0.9801 - val_loss: 0.0529 - val_accuracy: 0.9865
Epoch 4/10
28/28 [==============================] - 3s 94ms/step - loss: 0.0471 - accuracy: 0.9865 - val_loss: 0.0472 - val_accuracy: 0.9854
Epoch 5/10
35/35 [==============================] - 1s 16ms/step - loss: 0.0747 - accuracy: 0.9794
Epoch 1/10
28/28 [==============================] - 3s 104ms/step - loss: 0.2172 - accuracy: 0.9187 - val_loss: 0.1130 - val_accuracy: 0.9563
Epoch 2/10
28/28 [==============================] - 3s 94ms/step - loss: 0.0836 - accuracy: 0.9748 - val_loss: 0.0694 - val_accuracy: 0.9787
Epoch 3/10
35/35 [==============================] - 1s 16ms/step - loss: 0.1004 - accuracy: 0.9704
Epoch 1/10
28/28 [==============================] - 3s 104ms/step - loss: 0.2264 - accuracy: 0.9055 - val_loss: 0.0793 - val_accuracy: 0.9765
Epoch 2/10
28/28 [==============================] - 3s 94ms/step - loss: 0.0800 - accuracy: 0.9

Epoch 7/10
28/28 [==============================] - 3s 94ms/step - loss: 0.0682 - accuracy: 0.9804 - val_loss: 0.0697 - val_accuracy: 0.9798
Epoch 8/10
28/28 [==============================] - 3s 96ms/step - loss: 0.0664 - accuracy: 0.9795 - val_loss: 0.0694 - val_accuracy: 0.9798
Epoch 9/10
35/35 [==============================] - 1s 16ms/step - loss: 0.0789 - accuracy: 0.9794
Epoch 1/10
28/28 [==============================] - 3s 104ms/step - loss: 0.4872 - accuracy: 0.8037 - val_loss: 0.3532 - val_accuracy: 0.8711
Epoch 2/10
28/28 [==============================] - 3s 95ms/step - loss: 0.3126 - accuracy: 0.8710 - val_loss: 0.2072 - val_accuracy: 0.9383
Epoch 3/10
28/28 [==============================] - 3s 95ms/step - loss: 0.1561 - accuracy: 0.9560 - val_loss: 0.0971 - val_accuracy: 0.9686
Epoch 4/10
28/28 [==============================] - 3s 94ms/step - loss: 0.1005 - accuracy: 0.9731 - val_loss: 0.0862 - val_accuracy: 0.9709
Epoch 5/10
28/28 [==============================] - 3s

In [12]:
del model

NameError: name 'model' is not defined

In [15]:
model = None
with open('resultsRNN1reverse', 'w') as f:
    train_accuracies = np.zeros(5)
    test_accuracies = np.zeros(5)
    for fold in range(5):
        model = RNN_std()
        model.compile(loss=tf.keras.losses.binary_crossentropy ,optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),metrics=['accuracy'])
        history = model.fit(Etokseqrev[fold],L[fold],batch_size=128,epochs=10,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
        train_accuracies[fold] = history.history['accuracy'][-1]
        test_accuracies[fold] = model.evaluate(ETtokseqrev[fold],LT[fold])[1]
        del model
    mean_train = np.mean(train_accuracies)
    mean_test = np.mean(test_accuracies)
    string = "Accuracy on test: "+ mean_test + ", Accuracy on train: "+ mean_train + " \n"+ "Results obtained with RNN1 with bs = 128, lr = 0.0005, loss = BCE and optimizer = Adam on reverse mail messages"
    print(string)

Epoch 1/10
28/28 [==============================] - 3s 113ms/step - loss: 0.5595 - accuracy: 0.8427 - val_loss: 0.3998 - val_accuracy: 0.8711

TypeError: must be str, not numpy.float64